### Obtained Perplexity: -11.37, Coherence: 0.39, Best Number of Topics= 38

In [1]:
#Basic Libraries
import re
import numpy as np
import pandas as pd
from pprint import pprint
import sys
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy for lemmatization
import spacy
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

from topic_model_function import *

In [2]:
# NLTK Stop words
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to /Users/yxzh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#Load Dataset
import os
df=pd.read_csv('/Users/yxzh/PycharmProjects/GapAnalysis/Topic_based_Gap_Analysis/Anxiety/Data/CSV/Acad_2020.csv')
#getting rid of NaN
df=df.replace(np.nan, '', regex=True)
#Combining title and text
df["Text"] = df["Title"] + df["Abstract"]
#Now that we don't need Title or Text, we drop those columns before saving the file
df=df.drop(['Title', 'Abstract'], axis = 1)

In [4]:
df=df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [5]:
df.head()

,StoreId,documentType,year,Text
0,2.44597e+09,Journal Article,2020,Threat rapidly disrupts reward reversal learni...
1,2.44597e+09,Journal Article,2020,Fear in the context of pain: Lessons learned f...
2,2.44597e+09,Journal Article,2020,The effects of positive interpretation bias on...
3,2.43522e+09,"Evidence Based Healthcare , Journal Article",2020,Implementation and effectiveness of adolescent...
4,2.44597e+09,Journal Article,2020,The effects of age and trait anxiety on avoida...


In [6]:
# Convert to list
data=df.Text.values.tolist()
# Remove new line characters
data=[re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data=[re.sub("\'", "", sent) for sent in data]

In [7]:
#Function to clean up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) 

In [8]:
#Clean up text
data_words=list(sent_to_words(data))
print(data_words[:1])

[['threat', 'rapidly', 'disrupts', 'reward', 'reversal', 'learningthreat', 'changes', 'cognition', 'and', 'facilitates', 'adaptive', 'coping', 'however', 'when', 'threat', 'becomes', 'overwhelming', 'it', 'may', 'be', 'deleterious', 'to', 'mental', 'health', 'especially', 'for', 'vulnerable', 'individuals', 'flexible', 'decision', 'making', 'was', 'probed', 'with', 'reward', 'reversal', 'task', 'to', 'investigate', 'how', 'well', 'healthy', 'participants', 'can', 'adapt', 'to', 'changes', 'in', 'reward', 'contingency', 'when', 'they', 'expect', 'adverse', 'events', 'electric', 'shocks', 'in', 'comparison', 'to', 'safe', 'control', 'condition', 'the', 'threat', 'of', 'shock', 'significantly', 'impaired', 'reward', 'reversal', 'learning', 'moreover', 'enhanced', 'self', 'reported', 'threat', 'ratings', 'and', 'elevated', 'skin', 'conductance', 'levels', 'support', 'the', 'successful', 'induction', 'of', 'stressful', 'and', 'aversive', 'apprehensions', 'the', 'findings', 'are', 'in', 'lin

In [9]:
# Build the bigram and trigram models
bigram=gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram=gensim.models.Phrases(bigram[data_words], threshold=100)

In [10]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod=gensim.models.phrases.Phraser(bigram)
trigram_mod=gensim.models.phrases.Phraser(trigram)

In [11]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['threat', 'rapidly', 'disrupts', 'reward', 'reversal', 'learningthreat', 'changes', 'cognition', 'and', 'facilitates', 'adaptive', 'coping', 'however', 'when', 'threat', 'becomes', 'overwhelming', 'it', 'may', 'be', 'deleterious', 'to', 'mental', 'health', 'especially', 'for', 'vulnerable', 'individuals', 'flexible', 'decision_making', 'was', 'probed', 'with', 'reward', 'reversal', 'task', 'to', 'investigate', 'how', 'well', 'healthy', 'participants', 'can', 'adapt', 'to', 'changes', 'in', 'reward', 'contingency', 'when', 'they', 'expect', 'adverse_events', 'electric', 'shocks', 'in', 'comparison', 'to', 'safe', 'control', 'condition', 'the', 'threat', 'of', 'shock', 'significantly', 'impaired', 'reward', 'reversal', 'learning', 'moreover', 'enhanced', 'self', 'reported', 'threat', 'ratings', 'and', 'elevated', 'skin_conductance', 'levels', 'support', 'the', 'successful', 'induction', 'of', 'stressful', 'and', 'aversive', 'apprehensions', 'the', 'findings', 'are', 'in', 'line', 'with'

In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


In [13]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm")

# 1. Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

# 2. Create Dictionary needed for topic modelling
id2word = corpora.Dictionary(data_lemmatized)

# 3. Create Corpus
texts = data_lemmatized

# 4. Term Document Frequency and Create a bag of words
bow_corpus = bow(dictionary=id2word, processed_docs=texts)

# 5. Calculate low_tfidf_words
# Keep only words with tfidf ranking <= x * len(dictionary)
x = 0.2
total_word_count, DictDocFreq = tf_df(bow_corpus, id2word)
sorted_TFIDF = sort_tfidf(bow_corpus, total_word_count, DictDocFreq)
low_tfidf_words = get_low_tfidf_words(x, id2word, sorted_TFIDF)

# 6. Filter out least frequently used words
no_below = 0.01
keep_n = 10000
dict_least_freq_filtered = filter_least_frequent(id2word, texts, 
                                                 no_below, keep_n)

# 7. Filter out most commonly used words (i.e. words with low TF-IDF score)
dict_tfidf_filtered = filter_most_common(dict_least_freq_filtered, low_tfidf_words)

# 8. Create the second bag of words - bow_corpus_TFIDFfiltered, 
# created after least frequently and most commonly used words were filtered out.
corpus = bow(dict_tfidf_filtered, texts)

# View
[[(dict_tfidf_filtered[id], freq) for id, freq in cp] for cp in corpus[:1]]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dict_tfidf_filtered,
                                           num_topics=50,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

[['threat', 'rapidly', 'disrupt', 'reward', 'reversal', 'learningthreat', 'change', 'cognition', 'facilitate', 'adaptive', 'cope', 'however', 'threat', 'become', 'overwhelming', 'deleterious', 'mental', 'health', 'especially', 'vulnerable', 'individual', 'flexible', 'decision_making', 'probe', 'reward', 'reversal', 'task', 'investigate', 'well', 'healthy', 'participant', 'adapt', 'change', 'reward', 'contingency', 'expect', 'adverse_event', 'electric', 'shock', 'comparison', 'safe', 'control', 'condition', 'threat', 'shock', 'significantly', 'impair', 'reward', 'reversal', 'learn', 'moreover', 'enhanced', 'self', 'report', 'threat', 'rating', 'elevate', 'level', 'support', 'successful', 'induction', 'stressful', 'aversive', 'apprehension', 'finding', 'line', 'literature', 'show', 'stress', 'induce', 'inhibition', 'goal_directe', 'behavior', 'advantage', 'reflexive', 'habitual', 'response', 'style', 'notably', 'reversal', 'learning', 'rapidly', 'restore', 'omission', 'threat', 'several'

In [ ]:
# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=dict_tfidf_filtered, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df=pd.DataFrame()
    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row=row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row=sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j==0:  # => dominant topic
                wp=ldamodel.show_topic(topic_num)
                topic_keywords=", ".join([word for word, prop in wp])
                sent_topics_df=sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns=['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
     # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [ ]:
num_topics = 50
topic_words = []
for i in range(num_topics):
    tt = lda_model.get_topic_terms(i,10)
    topic_words.append([id2word[pair[0]] for pair in tt])

In [ ]:
for i in range(0,10):
    print(topic_words[i])

In [ ]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_lemmatized)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head()

In [ ]:
df_dominant_topic

In [ ]:
doc_lens = [len(d) for d in df_dominant_topic.Text]

# Plot
plt.figure(figsize=(16,7), dpi=160)
plt.hist(doc_lens, bins = 1000, color='navy')
plt.text(750, 100, "Mean   : " + str(round(np.mean(doc_lens))))
plt.text(750,  90, "Median : " + str(round(np.median(doc_lens))))
plt.text(750,  80, "Stdev   : " + str(round(np.std(doc_lens))))
plt.text(750,  70, "1%ile    : " + str(round(np.quantile(doc_lens, q=0.01))))
plt.text(750,  60, "99%ile  : " + str(round(np.quantile(doc_lens, q=0.99))))

plt.gca().set(xlim=(0, 1000), ylabel='Number of Documents', xlabel='Document Word Count')
plt.tick_params(size=16)
plt.xticks(np.linspace(0,1000,9))
plt.title('Distribution of Document Word Counts', fontdict=dict(size=22))
plt.show()

In [ ]:
# 1. Wordcloud of Top N words in each topic
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

cloud = WordCloud(stopwords=stop_words,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = lda_model.show_topics(formatted=False)

fig, axes = plt.subplots(4, 2, figsize=(10,10), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()

In [ ]:
df_dominant_topic.to_csv("Journal2020.csv")

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(alpha='auto', corpus=corpus, num_topics=num_topics, 
                                                id2word=dictionary, random_state=5)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.

model_list, coherence_values = compute_coherence_values(dictionary=dict_tfidf_filtered, corpus=corpus, 
                                                        texts=data_lemmatized, start=2, limit=100, step=8)

In [ ]:
# Show graph and compute the optimal number of topics

limit=100; start=2; step=8;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')

def differences(values, x):
    """
    Returns (optimal number of topics, largest jump, coherance value after jump)
    Params:
        values: list of coherance numbers, 
        x: num topics
    """
    x = list(x)
    max_diff = (0,0,0)
    for i in range(1,len(values)):
        print(x[i],values[i],values[i]-values[i-1])
        if values[i]-values[i-1] > max_diff[1]:
            max_diff = (x[i], values[i]-values[i-1], values[i])
    return max_diff

differences(coherence_values, x)

In [ ]:
# Show graph and compute the optimal number of topics

limit=100; start=2; step=8;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')

def differences(values, x):
    """
    Returns (optimal number of topics, largest jump, coherance value after jump)
    Params:
        values: list of coherance numbers, 
        x: num topics
    """
    x = list(x)
    max_diff = (0,0,0)
    for i in range(1,len(values)):
        print(x[i],values[i],values[i]-values[i-1])
        if values[i]-values[i-1] > max_diff[1]:
            max_diff = (x[i], values[i]-values[i-1], values[i])
    return max_diff

differences(coherence_values, x)